# Climate Ensemble Analysis - Studio Lab Quickstart

**Project**: Multi-Model Climate Projection Analysis  
**Domain**: Climate Science  
**Platform**: SageMaker Studio Lab (Free Tier)  

## What This Project Does

This notebook analyzes multiple climate models to create regional temperature projections. You'll learn how to:

- Access climate model data without downloads
- Calculate multi-model ensemble statistics
- Quantify uncertainty across models
- Create publication-quality figures
- Work with netCDF climate data

## Studio Lab Version Specifications

**Data**: 3 CMIP6 models (sample subset)  
**Region**: US Southwest  
**Variable**: Surface air temperature (tas)  
**Scenario**: SSP2-4.5 (moderate emissions)  
**Period**: 2015-2050  
**Runtime**: ~2-3 hours on CPU  
**Storage**: ~500MB  

## Prerequisites

- Basic Python knowledge
- Understanding of climate models (helpful but not required)
- Studio Lab account (free, no AWS account needed)

## Ready to Scale?

This Studio Lab version uses 3 models and pre-processed data. For production analysis with 20+ models and full CMIP6 access, see the [Unified Studio version](../unified-studio/).

---
## Setup & Imports

In [ ]:
# Check Python version
import sys
print(f"Python version: {sys.version}")
print(f"Expected: 3.10.x")

# Core libraries
import numpy as np
import pandas as pd
import xarray as xr

# Visualization
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns

# Utilities
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Progress bar
from tqdm.auto import tqdm

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("\n✅ All imports successful!")
print(f"\nKey library versions:")
print(f"  xarray: {xr.__version__}")
print(f"  numpy: {np.__version__}")
print(f"  pandas: {pd.__version__}")

---
## Configuration

Define analysis parameters. Modify these to explore different regions or time periods.

In [ ]:
# Models to analyze (Studio Lab uses 3 for speed)
MODELS = [
    'CESM2',           # Community Earth System Model 2 (NCAR)
    'GFDL-CM4',        # Geophysical Fluid Dynamics Lab CM4
    'UKESM1-0-LL'      # UK Earth System Model
]

# Variable
VARIABLE = 'tas'  # Surface air temperature

# Scenario
SCENARIO = 'ssp245'  # SSP2-4.5: Middle-of-the-road emissions

# Time period
START_YEAR = 2015
END_YEAR = 2050

# Region of interest: US Southwest
REGION = {
    'name': 'US Southwest',
    'lat_min': 31.0,
    'lat_max': 37.0,
    'lon_min': -114.0,
    'lon_max': -109.0
}

# Baseline period for anomalies
BASELINE_START = 1995
BASELINE_END = 2014

print(f"Configuration:")
print(f"  Models: {', '.join(MODELS)}")
print(f"  Variable: {VARIABLE} (surface air temperature)")
print(f"  Scenario: {SCENARIO}")
print(f"  Analysis period: {START_YEAR}-{END_YEAR}")
print(f"  Baseline period: {BASELINE_START}-{BASELINE_END}")
print(f"  Region: {REGION['name']}")
print(f"    Latitude: {REGION['lat_min']}°N to {REGION['lat_max']}°N")
print(f"    Longitude: {REGION['lon_min']}°E to {REGION['lon_max']}°E")

---
## Data Access

### For Studio Lab: Using Sample Data

In this Studio Lab version, we'll simulate climate model data for demonstration purposes. In the production Unified Studio version, you'll access real CMIP6 data directly from S3 without any downloads.

**Note**: This demonstrates the workflow with representative data. The actual values are simulated for educational purposes.

In [ ]:
def generate_sample_climate_data(model_name, region, start_year, end_year, baseline_start, baseline_end):
    """
    Generate sample climate data for demonstration.
    
    In production (Unified Studio), this would be:
    data = xr.open_dataset(f's3://cmip6-pds/CMIP6/.../tas_...{model_name}...nc')
    
    Args:
        model_name: Name of climate model
        region: Dictionary with lat/lon bounds
        start_year: Start year for analysis
        end_year: End year for analysis
        baseline_start: Baseline start year
        baseline_end: Baseline end year
    
    Returns:
        xarray.Dataset with temperature data
    """
    print(f"  Loading {model_name}...")
    
    # Create time coordinate (monthly data)
    full_start = baseline_start
    full_end = end_year
    time = pd.date_range(f'{full_start}-01-01', f'{full_end}-12-31', freq='MS')
    
    # Create spatial grid (1° resolution)
    lat = np.arange(region['lat_min'], region['lat_max'], 1.0)
    lon = np.arange(region['lon_min'], region['lon_max'], 1.0)
    
    # Generate realistic temperature data
    # Each model has different sensitivity and warming pattern
    model_params = {
        'CESM2': {'base_temp': 290.0, 'trend': 0.035, 'variability': 2.5},
        'GFDL-CM4': {'base_temp': 289.5, 'trend': 0.042, 'variability': 2.8},
        'UKESM1-0-LL': {'base_temp': 290.5, 'trend': 0.048, 'variability': 2.2}
    }
    
    params = model_params.get(model_name, model_params['CESM2'])
    
    # Create temperature array with:
    # - Base climatology
    # - Linear warming trend
    # - Seasonal cycle
    # - Interannual variability
    n_time = len(time)
    n_lat = len(lat)
    n_lon = len(lon)
    
    # Base temperature
    tas = np.ones((n_time, n_lat, n_lon)) * params['base_temp']
    
    # Add latitude gradient (warmer at southern latitudes)
    lat_gradient = np.linspace(5, -5, n_lat)
    tas += lat_gradient[np.newaxis, :, np.newaxis]
    
    # Add seasonal cycle
    month = np.array([t.month for t in time])
    seasonal = 10 * np.sin(2 * np.pi * (month - 1) / 12)
    tas += seasonal[:, np.newaxis, np.newaxis]
    
    # Add warming trend (years since start)
    years = np.array([(t.year - full_start) for t in time])
    trend = params['trend'] * years / 12  # Convert to monthly
    tas += trend[:, np.newaxis, np.newaxis]
    
    # Add interannual variability
    np.random.seed(hash(model_name) % 2**32)  # Reproducible per model
    variability = np.random.normal(0, params['variability'], (n_time, n_lat, n_lon))
    tas += variability
    
    # Create xarray Dataset
    ds = xr.Dataset({
        'tas': (['time', 'lat', 'lon'], tas)
    }, coords={
        'time': time,
        'lat': lat,
        'lon': lon
    })
    
    # Add metadata
    ds['tas'].attrs = {
        'long_name': 'Near-Surface Air Temperature',
        'units': 'K',
        'standard_name': 'air_temperature',
        'model': model_name,
        'scenario': SCENARIO
    }
    
    return ds

print("✅ Data loading function defined")

### Load All Model Data

In [ ]:
print("Loading climate model data...\n")

# Load data for each model
model_data = {}

for model in tqdm(MODELS, desc="Loading models"):
    ds = generate_sample_climate_data(
        model, 
        REGION, 
        START_YEAR, 
        END_YEAR,
        BASELINE_START,
        BASELINE_END
    )
    model_data[model] = ds

print(f"\n✅ Loaded {len(model_data)} models")
print(f"\nExample data structure (CESM2):")
print(model_data['CESM2'])

---
## Data Processing

### Calculate Regional Average Time Series

In [ ]:
def calculate_regional_mean(ds, region):
    """
    Calculate area-weighted regional mean.
    
    Args:
        ds: xarray Dataset with temperature data
        region: Dictionary with lat/lon bounds
    
    Returns:
        xarray DataArray with regional mean time series
    """
    # Extract temperature variable
    tas = ds['tas']
    
    # Calculate area weights (cosine of latitude)
    weights = np.cos(np.deg2rad(tas.lat))
    
    # Calculate weighted mean over lat/lon
    regional_mean = tas.weighted(weights).mean(['lat', 'lon'])
    
    return regional_mean

print("Calculating regional mean time series for each model...\n")

regional_means = {}
for model, ds in tqdm(model_data.items(), desc="Processing"):
    regional_means[model] = calculate_regional_mean(ds, REGION)

print("\n✅ Regional means calculated")
print(f"\nExample time series shape: {regional_means['CESM2'].shape}")
print(f"Time range: {regional_means['CESM2'].time.values[0]} to {regional_means['CESM2'].time.values[-1]}")

### Calculate Temperature Anomalies

Express temperatures as anomalies relative to the baseline period (1995-2014).

In [ ]:
def calculate_anomaly(time_series, baseline_start, baseline_end):
    """
    Calculate temperature anomaly relative to baseline period.
    
    Args:
        time_series: xarray DataArray with temperature data
        baseline_start: Start year of baseline period
        baseline_end: End year of baseline period
    
    Returns:
        xarray DataArray with temperature anomalies (K)
    """
    # Select baseline period
    baseline = time_series.sel(
        time=slice(f'{baseline_start}', f'{baseline_end}')
    )
    
    # Calculate baseline mean
    baseline_mean = baseline.mean('time')
    
    # Calculate anomaly
    anomaly = time_series - baseline_mean
    
    return anomaly

print("Calculating temperature anomalies...\n")

anomalies = {}
for model, ts in tqdm(regional_means.items(), desc="Calculating anomalies"):
    anomalies[model] = calculate_anomaly(ts, BASELINE_START, BASELINE_END)

print("\n✅ Anomalies calculated")
print(f"\nBaseline period: {BASELINE_START}-{BASELINE_END}")
print(f"Baseline mean (CESM2): {regional_means['CESM2'].sel(time=slice(str(BASELINE_START), str(BASELINE_END))).mean().values:.2f} K")

### Calculate Annual Means

Smooth monthly data to annual means for easier visualization.

In [ ]:
print("Calculating annual means...\n")

annual_anomalies = {}
for model, anom in tqdm(anomalies.items(), desc="Annual means"):
    # Resample to annual means
    annual = anom.resample(time='1Y').mean()
    annual_anomalies[model] = annual

print("\n✅ Annual means calculated")
print(f"\nAnnual data shape: {annual_anomalies['CESM2'].shape}")

---
## Ensemble Statistics

Calculate multi-model mean and uncertainty (spread).

In [ ]:
print("Calculating ensemble statistics...\n")

# Combine all models into single array
all_models = xr.concat(
    [annual_anomalies[m] for m in MODELS],
    dim='model'
)
all_models['model'] = MODELS

# Calculate ensemble mean
ensemble_mean = all_models.mean('model')

# Calculate ensemble spread (standard deviation)
ensemble_std = all_models.std('model')

# Calculate min/max range
ensemble_min = all_models.min('model')
ensemble_max = all_models.max('model')

print("✅ Ensemble statistics calculated")
print(f"\nEnsemble size: {len(MODELS)} models")
print(f"\nProjected warming by 2050 (relative to {BASELINE_START}-{BASELINE_END}):")

# Get 2050 values
year_2050 = annual_anomalies['CESM2'].sel(time='2050', method='nearest').time.dt.year.values

for model in MODELS:
    warming = annual_anomalies[model].sel(time='2050', method='nearest').values
    print(f"  {model}: {warming:.2f}°C")

mean_warming = ensemble_mean.sel(time='2050', method='nearest').values
std_warming = ensemble_std.sel(time='2050', method='nearest').values

print(f"\n  Ensemble mean: {mean_warming:.2f}°C")
print(f"  Ensemble spread (±1σ): ±{std_warming:.2f}°C")
print(f"  Likely range: {mean_warming-std_warming:.2f}°C to {mean_warming+std_warming:.2f}°C")

---
## Visualization

### Figure 1: Time Series with Uncertainty

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# Plot individual models (thin lines)
for model in MODELS:
    annual_anomalies[model].plot(
        ax=ax, 
        label=model, 
        linewidth=1.5, 
        alpha=0.6
    )

# Plot ensemble mean (thick line)
ensemble_mean.plot(
    ax=ax, 
    label='Ensemble Mean', 
    color='black', 
    linewidth=3
)

# Plot uncertainty range (shaded)
years = ensemble_mean.time.dt.year.values
ax.fill_between(
    ensemble_mean.time.values,
    (ensemble_mean - ensemble_std).values,
    (ensemble_mean + ensemble_std).values,
    color='gray',
    alpha=0.3,
    label='±1σ spread'
)

# Add zero line
ax.axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.5)

# Labels and title
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Temperature Anomaly (°C)', fontsize=12)
ax.set_title(
    f'Climate Projections for {REGION["name"]}\n'
    f'SSP2-4.5 Scenario | {len(MODELS)}-Model Ensemble',
    fontsize=14,
    fontweight='bold'
)

# Legend
ax.legend(loc='upper left', frameon=True, fancybox=True, shadow=True)

# Grid
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Figure 1: Time series with uncertainty")

### Figure 2: Model Agreement Analysis

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left panel: Warming rates by decade
decades = [
    ('2020s', '2020-2029'),
    ('2030s', '2030-2039'),
    ('2040s', '2040-2049')
]

decade_warming = {}
for decade_name, decade_range in decades:
    decade_data = []
    for model in MODELS:
        decade_mean = annual_anomalies[model].sel(
            time=slice(decade_range.split('-')[0], decade_range.split('-')[1])
        ).mean().values
        decade_data.append(decade_mean)
    decade_warming[decade_name] = decade_data

# Box plot
positions = np.arange(len(decades))
bp = ax1.boxplot(
    [decade_warming[d[0]] for d in decades],
    positions=positions,
    widths=0.6,
    patch_artist=True,
    showmeans=True,
    meanprops=dict(marker='D', markerfacecolor='red', markersize=8)
)

# Color boxes
for patch in bp['boxes']:
    patch.set_facecolor('lightblue')
    patch.set_alpha(0.7)

ax1.set_xticks(positions)
ax1.set_xticklabels([d[0] for d in decades])
ax1.set_ylabel('Temperature Anomaly (°C)', fontsize=11)
ax1.set_title('Decadal Mean Warming', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Right panel: Model spread over time
ensemble_std.plot(ax=ax2, color='darkred', linewidth=2)
ax2.set_xlabel('Year', fontsize=11)
ax2.set_ylabel('Inter-Model Std Dev (°C)', fontsize=11)
ax2.set_title('Model Uncertainty Over Time', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Figure 2: Model agreement analysis")

---
## Summary Statistics

In [ ]:
print("=" * 70)
print("CLIMATE PROJECTION SUMMARY")
print("=" * 70)
print(f"\nRegion: {REGION['name']}")
print(f"Scenario: {SCENARIO} (SSP2-4.5)")
print(f"Models: {', '.join(MODELS)}")
print(f"Baseline period: {BASELINE_START}-{BASELINE_END}")

print(f"\n{'Decade':<15} {'Mean Warming (°C)':<20} {'Model Range (°C)':<25}")
print("-" * 70)

for decade_name, decade_range in decades:
    start, end = decade_range.split('-')
    
    # Get decade means for all models
    decade_means = []
    for model in MODELS:
        dm = annual_anomalies[model].sel(
            time=slice(start, end)
        ).mean().values
        decade_means.append(dm)
    
    mean_val = np.mean(decade_means)
    min_val = np.min(decade_means)
    max_val = np.max(decade_means)
    
    print(f"{decade_name:<15} {mean_val:>7.2f} ± {np.std(decade_means):4.2f}       {min_val:>6.2f} to {max_val:>5.2f}")

print("\n" + "=" * 70)
print("KEY FINDINGS:")
print("=" * 70)

warming_2050 = ensemble_mean.sel(time='2050', method='nearest').values
spread_2050 = ensemble_std.sel(time='2050', method='nearest').values

print(f"\n1. Projected warming by 2050: {warming_2050:.2f}°C ± {spread_2050:.2f}°C")
print(f"   (relative to {BASELINE_START}-{BASELINE_END} baseline)")

# Calculate rate of warming
early = ensemble_mean.sel(time=slice('2015', '2025')).mean().values
late = ensemble_mean.sel(time=slice('2040', '2050')).mean().values
rate = (late - early) / 25  # Per decade

print(f"\n2. Rate of warming: {rate*10:.2f}°C per decade")

# Model agreement
agreement = (ensemble_std.sel(time='2050', method='nearest').values / 
             abs(ensemble_mean.sel(time='2050', method='nearest').values)) * 100

print(f"\n3. Model uncertainty: {agreement:.1f}% (relative to ensemble mean)")

if agreement < 30:
    print("   → High confidence (models agree well)")
elif agreement < 50:
    print("   → Medium confidence (moderate model spread)")
else:
    print("   → Lower confidence (large model spread)")

print(f"\n4. All {len(MODELS)} models project warming in this region")
print("   → Robust signal across models")

print("\n" + "=" * 70)

---
## Next Steps

### Experiments to Try

1. **Change the region**: Modify `REGION` dictionary to analyze a different area
2. **Different time period**: Adjust `START_YEAR` and `END_YEAR`
3. **Add more visualizations**: Create seasonal analysis, spatial patterns

### Ready to Scale?

This Studio Lab version uses 3 models and simulated data. For production research:

**Unified Studio Version Includes**:
- 20+ CMIP6 models (real data from S3)
- Full spatial resolution
- Multiple scenarios (SSP1-2.6, SSP2-4.5, SSP3-7.0, SSP5-8.5)
- Multiple variables (temperature, precipitation, etc.)
- Distributed processing with EMR
- Bedrock AI for report generation
- Publication-quality figures
- Statistical significance testing

**Cost**: ~$20-30 for full analysis  
**Time**: 1-2 days  
**Value**: Months of traditional work → Days  

[View Unified Studio Version →](../unified-studio/)

### Learn More

- [Transition Guide](../../../../docs/transition-guides/studio-lab-to-unified.md)
- [Platform Comparison](../../../../docs/getting-started/platform-comparison.md)
- [FAQ](../../../../docs/resources/faq.md)

### Contribute

Found this useful? Consider:
- ⭐ Star the [GitHub repo](https://github.com/research-jumpstart/research-jumpstart)
- 💬 Share your experience in [Discussions](https://github.com/research-jumpstart/research-jumpstart/discussions)
- 🐛 Report issues or suggest improvements
- 🚀 Contribute your own project

---

**Research Jumpstart** | Built by researchers, for researchers 🚀